# Introduction

Copyright

Distribution

In order to use the code the following libraries, classes and functions must be installed:
1. Pandas
2. Numpy
3. RegEx
4. D-wave Samplers as below
5. Dimod modules as below

In [ ]:
import pandas as pd
import numpy as np
from dwave.system.samplers import DWaveSampler
from dwave.system import LeapHybridCQMSampler
from dwave.system.composites import EmbeddingComposite
from dimod import quicksum
from dimod import Binary
from dimod import CQM
import dwave.inspector
import dwavebinarycsp
import re
from time import process_time
import pprint

# Input and Preprocessing (Anna Ehrenberg)

In this section all data input is entered, validated and preprocessed in order to meet further requirements. 

In this section the following input is proceeded:

Input data file with information on ferry routes with
- From: route start port (Column1)
- To: route end port (Row1)
- Distance: shortest direct distance between departure and arrival port (Number in matrix). If the connection is not possible, the distance is zero or no value is entered.


### Input data file

In [ ]:
df_ports = pd.read_csv("data_harbours_utf8.csv", sep = ";")
df_ports

### Formatting of data

In order to formulate the problem in a format accessible by the D-Wave libraries the matrix data is restructured into an unstacked format. The new table contains of the follwoing columns:
1. From
2. To
3. Distance
After tranforming the format, only possible routes are included in the data.

In [ ]:
# rename 'Column1' to 'From'
df_ports=df_ports.rename(columns={'Column1':"From"})

# transform end harbours in columns to rows
df_routes= pd.melt(df_ports,id_vars=['From'],var_name="To",value_name='Distance')

# filter NaN-values
df_routes=df_routes[df_routes["Distance"].notnull()]

# filter out 0.0 values
df_routes=df_routes[df_routes.Distance != 0]

#reset index
df_routes=df_routes.reset_index(drop=True)
df_routes_withoutStardEnd=df_routes
df_routes

### User input and validation of departure and destination harbour 

In this section the manual user input is requested and validated by the following criteria:
1. departure and destination ports exist in the input file with connections
2. departure port is not equal to destination port

If input is invalid, the according error message is displayed.

In [ ]:
#Check whether the departure port is element of the input data
while True:
    #Entering departure port
    departure=input(str("Please enter departure port:"))
    if not departure in df_routes['From'].values:
        print("Entered port is not defined as a depature port in the input file. Please enter valid departure port.")
        continue
    else:
        break

#Check whether the destination port is element of the input data and not equal to departure port
while True:
    #Entering destination port
    destination=input(str("Please enter destination port:"))
    if not destination in df_routes['To'].values:
        print("Entered port is not defined as a destination port in the input file. Please enter destination valid port.")
        continue
    if destination == departure:
        print("Entered destination port is equal to departure port. Please enter valid destination port.")
        continue
    else:
        #we're happy with the value given.
        break


# Data Preparation for Quantum Computing (Anna Ehrenberg, Nick Stuke)

In this section, all methods are defined that are later used to
1. create Contraint Satisfaction Problem (CSP)
2. formulate Binary Quadratic Model (BQM) as QUBO. 

A problem can be formulated in different ways in order to provide the quantum computer with it. In this example, the problem is formulated as a BQM in the form of a QUBO. The mathematical phrase can be attained by a support class called CSP or can directly be input. In this case, the problem will first be represented by a CSP. The CSP constitutes of 
1. all variables and 
2. constraints on the variables.

### Basic functions

In order to simplify code in the constraints section, the following functions are defined. 

In [ ]:
def sum_to_two_or_zero(*args):
        """Checks to see if the args sum to either 0 or 2.
        """
        sum_value = sum(args)
        return sum_value in [0, 2]
        
def sum_smaller_equal_one(*args):
        """Checks to see if the args sum to either smaller or equal to 1.
        """
        sum_value = sum(args)
        return sum_value in [0, 1]

def get_labels(dataframe):
        """Returns a list of labels from a Dataframe of the format of the input file"""
        labels=(dataframe['From']+'-'+dataframe['To']+'-'+dataframe['Distance'].astype(str)).values.tolist()
        return labels

### Definition class Ferryarea

The class Ferryarea is defined as the geographical space in which the routes connect the ports.
Each Ferryarea has a start and end point as well as ports and routes between those ports.

In [ ]:
class Ferryarea:
    def __init__(self,start,end,routes):
        """ Initializes object of Type Ferryarea"""
        #Instantiate
        self.start=start
        self.end=end
        self.df_routes = routes
        self.ports = pd.DataFrame({'ports':np.append(df_routes['To'].unique(),df_routes['From'].unique())}).drop_duplicates().values.tolist()
        self.csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        self.cqm = CQM()
        self.objectiveDistances=0
        #Add start and end route
        self.df_routes= self.df_routes.append({'From':self.end, 'To':'end','Distance':0}, ignore_index=True)
        self.df_routes= self.df_routes.append({'From':'start', 'To':self.start,'Distance':0}, ignore_index=True)
        
    def _get_csp(self):
        return self.csp
    def _get_cqm(self):
        return self.cqm    
    def _apply_valid_routes_constraint(self):
        """Creates constraints defining that every entered port has to be exited as well and that every port can only be entered and exited through one route."""

        for port in self.ports:
            #assign to directions Dataframe a Dataframe containing only direction to and from the port
            df_directions_from= self.df_routes[self.df_routes['From']==port[0]]
            df_directions_to= self.df_routes[self.df_routes['To']==port[0]]
            df_directions= pd.concat([df_directions_to,df_directions_from])
            
            #creates sets of labels as basis for constraint defintion
            directions_all = set(get_labels(df_directions))
            
            #set((df_directions['From']+'-'+df_directions['To']+'-'+df_directions['Distance'].astype(str)).values.tolist())
            directions_to= set(get_labels(df_directions_to))
            
            #set((df_directions_to['From']+'-'+df_directions_to['To']+'-'+df_directions_to['Distance'].astype(str)).values.tolist())
            directions_from= set(get_labels(df_directions_from))
            #set((df_directions_from['From']+'-'+df_directions_from['To']+'-'+df_directions_from['Distance'].astype(str)).values.tolist())
              
            x={r: Binary(
                "%r" %r
            )for r in directions_all}

            for d in directions_all:
                distance=float(d.split('-')[2])
                self.objectiveDistances=self.objectiveDistances+distance*x[d]
            
            self.cqm.add_constraint(
                quicksum([x[tod] for tod in directions_to])
                -
                quicksum([x[fromd] for fromd in directions_from]) 
                == 0)
            
            #add constraint sum_to_two_or_zero
            self.csp.add_constraint(sum_to_two_or_zero,directions_all)

            #add constraint 'To' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one,directions_to)
            
            #add constraint 'From' sum_to_zero_or_one
            self.csp.add_constraint(sum_smaller_equal_one, directions_from)

    def _set_CQM_objective(self):
        self.cqm.set_objective(self.objectiveDistances)
        
    def _set_start_and_end(self):
        """Sets the values of the departure and destination port to 1 and all other start and end possibilities to 0
        """ #all other possibilities do not have to be 0. However, this is expected to descrease calculation time

        #select all start and end routes
        df_routes_to_fix= pd.concat([self.df_routes[self.df_routes['From']=='start'],self.df_routes[self.df_routes['To']=='end']])
        count=0

        for i in df_routes_to_fix.values:
            label= get_labels(df_routes_to_fix.iloc[[count]])
            if (df_routes_to_fix['To'].iloc[count])==self.start:
                #sets departure to 1
                self.csp.fix_variable(label[0],1)
            elif (df_routes_to_fix['From'].iloc[count])==self.end:
                #sets destination to 1
                self.csp.fix_variable(label[0],1)
            else:
                #sets departure and destination to 0
                self.csp.fix_variable(label[0],0)
            count=count+1

        self.cqm.fix_variable("'start-"+departure+"-0.0'",1)
        self.cqm.fix_variable("'"+destination+"-end-0.0'",1)
       

### Initialization of Ferryarea

In this section, a Ferryarea is initialized with departure and destination port and the routes from the input file. It is used to create the models that are later handed to the Quantum Samplers. 

There are two Samplers used in this case:

1. Quantum Sampler: This sampler only computes on the quantum computer. It requires a binary quadratic model as input. In this case, the BQM is built by a Constraint Satisfaction Problem. This is a model provided by dwave in order to built a BQM. First, all variables and constraints must be added. The variables in this example will represent the routes between two ports. For example, the route Port1-Port2-Distance12 can be used (1) or not (0). 
A set of support variables is needed in order to represent the entered departure and destination port. Therefor, artificial routes from "start" port to the departure port and to "end" port from the destination port are created in the following code. 

2. Hybrid Sampler: The Hybrid Sampler is a blend of quantum computer and a classical computer. It requires a constraint quadratic model (CQM) as input. Compared to the BQM, it uses integers instead of binary values. Similar to the BQM, constraints are added and variables such as the start and end are fixed. Additionally, an objective is defined. In this case, the objective is to minimize the traveled distance between start and end port.

In [ ]:
f=Ferryarea(start=departure,end=destination,routes=df_routes)

Two constraints ensure, that the problem is displayed correctly:
1. All routes connected to one port have to sum up to 0 or 2. This ensures that every port that is entered is exited as well.
2. The routes are one-way. This means it is possible to go from one port to another but not back the same way. With the first constraint is would be possible to take two routes where the port is listed in the "from" column. This means, it would be exited twice but never approached. Thus, another constraint is needed to ensure that ports are approached only once and existed only once. 

In [ ]:
f._apply_valid_routes_constraint()

After all constraints are applied, start and end routes are fixed to 1. 

In [ ]:
f._set_start_and_end()

In addition, the objective for the CQM is defined.

In [ ]:
f._set_CQM_objective()

The models are created as follows. 

In [ ]:
csp= f._get_csp()
cqm=f._get_cqm()

While the CQM can be processed directly by the Hybrithe CSP must first be transformed into a BQM. In order to fit the Problem to the modell it is necessary to adjust the following parameters:
- max_graph_size: This parameters limits the number of variables that can be included in one constraint. In this case, the number of connections to and from one port. The original d-wave function dwavebinarycsp.stitch uses the function np.unpackbits from the numpy library. This only unpacks arrays from the size of 8 bits. Thus, the function all_possible() and the dtype of the matrix A in the file generation.py have been modified. Now, up to 32 connections between variables can be entered. 

In [ ]:
# Create BQM, max_graph_size gets bigger once more variables are included in one constraint
import multiprocessing
import time

csp_stitchTimeout=False
def stichFunction():
    bqm = dwavebinarycsp.stitch(csp,max_graph_size=15)

p = multiprocessing.Process(target=stichFunction, name="StichFunction")
p.start()

# Timeout in x seconds seconds
p.join(10)


if p.is_alive():
    print "Stich-Function runs too loo long"
    # Terminate StitchFunction
    p.terminate()
    p.join()
    csp_stitchTimeout=Tru

### Results BQM

The BQM is created by the stitch() function provided by d-wave. However, the larger the input data gets the longer is takes for the stitch() function to built the BQM. Within the function a graph is built with all variables. Using the sample data, a graph of 9 nodes and 36 edges is created and the BQM can be built in approximately 42 seconds. For a graph with 10 nodes, approximiately 15 minutes are needed.
In conclusion, it is not practical to use the BQM model for real world problems. 

# Transfer to Quantum Sampler (Anna Ehrenberg, Nick Stuke)

In this section, the before built models are tranferred to the quantum computer by using D-wave's Samplers.

## BQM-Sampling and Result-Selection (Anna Ehrenberg, Nick Stuke)

So far, the model can find possible ways between two ports. In the following code, a penalty is added in order to penalize long distances. This way, the shortest route is selected. However, the number of stops is not taken into consideration. 

In [ ]:
if not csp_stitchTimeout:
    # Add Penalty
    penalty=0.01
    for v in bqm.variables:
        # Ignore auxiliary variables
        if isinstance(v, str) and re.match(r'^aux\d+$', v):
            continue
        split_v = v.split('-')
        bqm.add_variable(v, penalty*float(split_v[2]))

The sampler for BQMs is assigned.

In [ ]:
if not csp_stitchTimeout:
    # define Sampler
    sampler = EmbeddingComposite(DWaveSampler())

The sampler is called with the input BQM, number of reads and chain strengths. All results are saved with the produced energy. The result with the lowest energy is expected to be the best result. 

In [ ]:
if not csp_stitchTimeout:
    # run BQM on QPU
    result = sampler.sample(bqm,
                        num_reads=100,
                        chain_strength=3,
                        label='ferrylines')

The results can be visualized online with the D-wave inspector.

In [ ]:
if not csp_stitchTimeout:
    # visualize result
    dwave.inspector.show(result)

The final result is displayed.

In [ ]:
if not csp_stitchTimeout:
    print(result.first.sample)

## CQM-Sampling and Result-Selection (Nick Stuke)

In [ ]:
sampler_cqm = LeapHybridCQMSampler()
sample_set_cqm=sampler_cqm.sample_cqm(cqm)

In [ ]:
n_samples = len(sample_set_cqm.record)
feasible_samples = sample_set_cqm.filter(lambda d: d.is_feasible) 

In [ ]:
best_feasible_cqm_solution=DWaveSampler.sample
if not feasible_samples: 
    raise Exception("No feasible solution could be found for this problem instance.")
else:
    best_feasible_cqm_solution = feasible_samples.first
    print(best_feasible_cqm_solution)

# Shortest Way with classical Computing (Nick Stuke)

This Chapter show an algorithm on a classical computer with no quantum-computer ressources. A common algorithm for the given problem is the Dijkstra-Algorthm\[1\](vgl. S.1).
As a Greedy-Algorithm Dijkstra processes each node once, so with more nodes, it needs more process-time.
A* is a much faster algorithm but don't find always the optimal solution.\[2\](vgl. S.2)
Since the quantum computer should have exactly the strengths of quickly finding global minima, the Dijkstra algorithm is used in this notebook.

## Define Graph
First of all, we have to create a graph. Therefore we iterate over every port and add the routes away from it to the graph-note. The graph is stored as as dictonary.

In [41]:
# define Graph with distances for every Point
graph={}
for index,port in df_ports.iterrows():
    routes=df_routes_withoutStardEnd[df_routes_withoutStardEnd['From']==port['From']]
    distances={}
    for i,r in routes.iterrows():
        distances[r['To']] = r['Distance']
    graph[port['From']] =distances
print(graph)


{'Bremerhaven': {'Emden': 137.0, 'Kiel': 135.0, 'Aberdeen': 435.0}, 'Brunsbüttel': {'Bremerhaven': 81.0, 'Hamburg': 36.0, 'Kiel': 54.0, 'Rotterdam': 269.0, 'Aarhus': 187.0, 'Copenhagen': 216.0, 'Bornholm': 223.0, 'Gdansk': 398.0, 'Riga': 604.0}, 'Emden': {'Bremerhaven': 137.0}, 'Hamburg': {'Bremerhaven': 117.0, 'Brunsbüttel': 36.0, 'Kiel': 90.0, 'Lübeck': 187.0, 'Rostock': 174.0, 'Sassnitz': 145.0, 'Wilhelmshaven': 114.0, 'Wismar': 176.0, 'Antwerpen': 405.0, 'Rotterdam': 305.0}, 'Kiel': {'Bremerhaven': 135.0, 'Stralsund': 109.0, 'Rotterdam': 323.0, 'Stockholm': 497.0}, 'Lübeck': {'Stralsund': 93.0}, 'Rostock': {}, 'Sassnitz': {'Kiel': 145.0, 'Rostock': 92.0, 'Aarhus': 204.0, 'Copenhagen': 97.0, 'Bornholm': 52.0, 'Gdansk': 213.0, 'Klaipeda': 271.0, 'Riga': 497.0, 'Tallinn': 497.0}, 'Stralsund': {}, 'Wilhelmshaven': {}, 'Wismar': {'Bremerhaven': 585.0, 'Brunsbüttel': 140.0, 'Hamburg': 176.0, 'Kiel': 86.0, 'Rotterdam': 713.0, 'Aarhus': 160.0, 'Copenhagen': 139.0, 'Bornholm': 145.0, 'Gdans

## Calculate shortest distance to start-port


In [ ]:
queue = [departure]
d = {node: {"shortest distance":float("inf"), "previous":None} for node in graph}
d[departure]["shortest distance"] = 0
cpuTime_start=process_time() 
while queue:
    current = queue.pop(0)
    shortest_distance = d[current]["shortest distance"]
    for neighbour in graph[current]:

        dist_to_neighbour = graph[current][neighbour]

        if shortest_distance + dist_to_neighbour < d[neighbour]["shortest distance"]:

            d[neighbour] = {
                "shortest distance": shortest_distance + dist_to_neighbour,
                "previous": current
            }
            queue.append(neighbour)
cpuTime_end=process_time()
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(d)
print()

## Create Path with shortest distances

In [ ]:
current = destination
print('Destination: '+str(destination))
print('Departure: '+str(departure))
dji_path = [current]
distance_sum=0.0
while current != departure:
    print(current)
    current = d[current]["previous"]
    dji_path.append(current)

dji_path = dji_path[::-1]
print('Distance: '+str(d[destination]["shortest distance"])+' Pfad: '+str(dji_path))

# Result-Consideration (Nick Stuke)

## Result-Homogenization

In the following area, a brief processing of the results is carried out. This is required for easier presentation and does not change or add any information.

In [ ]:
result_csp=[departure]
result_cqm=[departure]
result_dji=dji_path
cpuTime_dji=cpuTime_end-cpuTime_start

# homogenization of cqm-solution
solution=[]
for s in best_feasible_cqm_solution.sample:
    if (best_feasible_cqm_solution.sample[s]==1.0 and 
     s.find('end')==-1 and 
     s.find('start')==-1):
        solution.append(s)
currentP=departure
while currentP!=destination:
    for s in solution:
        s=s.replace("'","").split('-')
        if s[0].find(currentP)==0:
            currentP=s[1]
            result_cqm.append(s[1])
            break

# homogenization of csp-solution
solution=[]
for s in result.first.sample:
    if (result.first.sample[s]==1 and 
     s.find('end')==-1 and 
     s.find('start')==-1):
        solution.append(s)
currentP=departure
while currentP!=destination:
    for s in solution:
        s=s.split('-')
        if s[0].find(currentP)==0:
            currentP=s[1]
            result_csp.append(s[1])
            break


## Result-Summary

In the following, the results are presented in the form of a route from the start to the end destination. Which algorithm each belongs to is included in the Python output.


In [ ]:
print('result-cqm: '+str(result_cqm))
print('result-csp: '+str(result_csp))
print('result-dji: '+str(result_dji))
print('time-dji: '+str(cpuTime_dji))

On a small set of data, all three algorithms were able to find the correct solution. Djikstra is considered a comparison here, since the solution is calculated individually and is therefore seen as correct. The calculation times are irrelevant.

The situation is different with a larger data set. With the CSP, the stitch function took too long and was therefore aborted. The reason for the long execution time has already been explained in the execution of the sting function. The other two algorithms were able to calculate the route with negligible time.

# Conclusion and chances

# Quellen

\[1\] Broumi, Said, Assia Bakal, Mohamed Talea, Florentin Smarandache, und Luige Vladareanu. „Applying Dijkstra algorithm for solving neutrosophic shortest path problem“. In 2016 International Conference on Advanced Mechatronic Systems (ICAMechS), 412–16. Melbourne, Australia: IEEE, 2016. https://doi.org/10.1109/ICAMechS.2016.7813483.

\[2\] Candra, Ade, Mohammad Andri Budiman, und Kevin Hartanto. „Dijkstra’s and A-Star in Finding the Shortest Path: a Tutorial“. In 2020 International Conference on Data Science, Artificial Intelligence, and Business Analytics (DATABIA), 28–32. Medan, Indonesia: IEEE, 2020. https://doi.org/10.1109/DATABIA50434.2020.9190342.
